In [1]:
manual_compile = True

In [2]:
import pandas as pd
import os
from datetime import datetime

In [3]:
import utils.utils_var as utils_var

In [4]:
from config.config_loader import Config

config = Config()

In [5]:
path_facturas_elec = config.get_resolved_path("data_path.elec.processed.facturas")
path_facturas_elec_manual = config.get_resolved_path("data_path.elec.processed.facturas_manual")
customers_path = config.get_resolved_path("data_path.elec.customers.folder")
path_alias_elec = config.get_resolved_path("data_path.elec.customers.alias_elec")

In [6]:
clientes_elec = os.listdir(customers_path)
clientes_elec

['alias_clientes_elec - copia.xlsx',
 'alias_clientes_elec.xlsx',
 'Altan Pharma',
 'Catalonia Fundacio',
 'Generic',
 'Grupo Inspired',
 'Kem One',
 'Montfrisa',
 'PR',
 'PR2',
 'Sundolitt',
 'Taghleef',
 'Trilla y Famesa',
 'VYOSA']

In [7]:
df_alias = pd.read_excel(path_alias_elec)
df_alias.tail()

,cliente,alias,cups,tarifa,electrointensivo,contrato
105,Catalonia Fundacio,EDDA DE CARLI,ES0031408015307001CW,2.0 TD,False,1
106,Catalonia Fundacio,BESORA,ES0031405265052002WH,2.0 TD,False,1
107,Catalonia Fundacio,S JORDI 2,ES0031405060747021FN,2.0 TD,False,1
108,Catalonia Fundacio,HOSPITALET,ES0031406166448011FZ,2.0 TD,False,1
109,Generic,TRIAL,TRIAL,6.1 TD,False,1


## Extraccion de facturas

In [8]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(f'sqlite:///{path_facturas_elec}', echo=True)
existing_data = pd.read_sql(('SELECT * FROM facturas'), con=engine)

2025-07-02 10:05:42,918 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-02 10:05:42,918 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")
2025-07-02 10:05:42,918 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-02 10:05:42,934 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")
2025-07-02 10:05:42,934 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-02 10:05:42,934 INFO sqlalchemy.engine.Engine SELECT * FROM facturas
2025-07-02 10:05:42,934 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-02 10:05:42,962 INFO sqlalchemy.engine.Engine ROLLBACK


In [9]:
existing_data.columns

Index(['num_factura', 'inicio_periodo', 'fin_periodo', 'issue_date', 'cups',
       'coste_potencia', 'coste_energia', 'excesos_potencia',
       'dto_electrointensivo', 'coste_reactiva', 'total_bruto_ie_iva',
       'total_bruto_iva', 'cliente', 'alias', 'potencia_p1', 'potencia_p2',
       'potencia_p3', 'potencia_p4', 'potencia_p5', 'potencia_p6',
       'energia_p1', 'energia_p2', 'energia_p3', 'energia_p4', 'energia_p5',
       'energia_p6', 'maximetros_p1', 'maximetros_p2', 'maximetros_p3',
       'maximetros_p4', 'maximetros_p5', 'maximetros_p6', 'year', 'month'],
      dtype='object')

In [10]:
existing_data['cliente'].unique()

array(['PR', 'Catalonia Fundacio', 'Kem One', 'Grupo Inspired',
       'Sundolitt', 'Trilla y Famesa', 'Altan Pharma', 'Taghleef',
       'Montfrisa', 'VYOSA'], dtype=object)

In [12]:
if manual_compile:

    factura_list = []
    i = 0
    try:
        df_sample = pd.read_pickle(path_facturas_elec_manual)
    except:
        df_sample = pd.DataFrame()

    for concepto in existing_data.columns:

        if 1 <= i <= 3:
            concept = str(input(f'Introduce {concepto}: '))
            concept = datetime.strptime(concept, '%d.%m.%Y').date()
        elif i == 0 or i == 4:
            concept = str(input(f'Introduce {concepto}: '))
        elif 5 <= i <= 11 or 14 <= i <= 31:#25:
            concept = input(f'Introduce {concepto}: ')
            try:
                concept = float(concept)
            except:
                concept = str(concept)
                new_concept = ''
                for s in concept:
                    if s == ',':
                        r = '.'
                        new_concept+=r
                    elif s != '.':
                        new_concept+=s
                concept = float(new_concept)
        else:
            concept = 0
        
        print(concept)
        factura_list.append(concept) 
        i+=1

    factura_df = pd.DataFrame([factura_list], columns=existing_data.columns)
    df_manual = pd.concat([df_sample, factura_df])
    df_manual.drop_duplicates(inplace=True)
    df_manual.reset_index(drop=True, inplace=True)

    df_manual['month'] = df_manual['fin_periodo'].apply(lambda x: x.month)
    df_manual['year'] = df_manual['fin_periodo'].apply(lambda x: x.year)
    df_manual['cliente'] = df_manual.apply(
        lambda row: utils_var.insert_alias(cups=row['cups'], df_alias=df_alias)[0], axis=1)
    df_manual['alias'] = df_manual.apply(
        lambda row: utils_var.insert_alias(cups=row['cups'], df_alias=df_alias)[1], axis=1)
    df_manual.to_pickle(path_facturas_elec_manual)

    df_manual.tail()
    # ES0021000003225511XB
    # cambiar energia en noviembre


P25CON026585364
2025-05-01
2025-05-31
2025-06-05
ES0021000021881819JF
5774.99
31399.03
0.0
0.0
0.0
37174.42
39075.03
0
0
1150.0
1150.0
1150.0
1150.0
1150.0
2000.0
0.0
0.0
0.0
126896.0
83058.0
143313.0
0.0
0.0
0.0
1076.0
1008.0
1060.0
0
0


In [ ]:
if not manual_compile:
    df_manual = pd.read_pickle(path_facturas_elec_manual)

print(df_manual.tail(6)['cups'])

# df_manual.loc[df_manual['num_factura'] == '03241205010000023', 'coste_energia'] = 3117.75
# df_manual['cliente'] = df_manual.apply(
#     lambda row: utils_var.insert_alias(cups=row['cups'], df_alias=df_alias)[0], axis=1)
# df_manual['alias'] = df_manual.apply(
#     lambda row: utils_var.insert_alias(cups=row['cups'], df_alias=df_alias)[1], axis=1)

# df_manual.to_pickle(path_facturas_elec_manual)
df_manual.tail(6)


37    ES0031500694885001LS
38    ES0031500694885001LS
39    ES0031500694885001LS
40    ES0022000008103260DW
41    ES0021000003225511XB
42    ES0022000009138572BS
Name: cups, dtype: object


,num_factura,inicio_periodo,fin_periodo,issue_date,cups,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,...,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6,year,month
37,P25CON010473271,2025-02-01,2025-02-28,2025-03-06,ES0031500694885001LS,476.87,2491.46,1.22,0.0,0.0,...,0.0,4053.0,0.0,0.0,120.0,144.0,0.0,92.0,2025,2
38,P25CON015524126,2025-03-01,2025-03-31,2025-04-05,ES0031500694885001LS,527.98,2305.34,0.00,0.0,0.0,...,5209.0,4344.0,0.0,0.0,0.0,128.0,120.0,64.0,2025,3
39,P25CON020865917,2025-04-01,2025-04-30,2025-05-05,ES0031500694885001LS,510.92,1655.09,0.00,0.0,0.0,...,3528.0,3750.0,0.0,0.0,0.0,92.0,112.0,60.0,2025,4
40,218169350,2025-04-01,2025-04-30,2025-06-04,ES0022000008103260DW,6055.36,11908.87,0.00,0.0,0.0,...,34147.0,90115.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,4
41,03250605010000019,2025-05-01,2025-05-31,2025-06-05,ES0021000003225511XB,5583.18,1117.86,0.75,0.0,0.0,...,172222.0,427057.0,0.0,0.0,0.0,1684.0,1744.0,1807.0,2025,5
42,218178732,2025-05-01,2025-05-31,2025-06-17,ES0022000009138572BS,12559.46,38326.63,0.00,0.0,0.0,...,135344.0,299998.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,5
